In [1]:
# Load libraries
from scipy.io import arff
import urllib.request
import io
import pandas as pd
import seaborn as sns
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from hashlib import sha256, blake2b, algorithms_available

In [9]:
#Código para cargar el Dataset
url = "sabana_sede1.csv"
df_origin = pd.read_csv(url, delimiter = ",", on_bad_lines="skip", encoding='unicode_escape', dtype = {'SEDE':'str', 'HABITACION_ATENCION':'str'})
df_origin.info()

C:\Users\jefe.sistemas\AppData\Local\Temp\ipykernel_27384\2155077019.py:3: DtypeWarning: Columns (43,44,45,46,47,48,49,50,51,52,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df_origin = pd.read_csv(url, delimiter = ",", on_bad_lines="skip", encoding='unicode_escape', dtype = {'SEDE':'str', 'HABITACION_ATENCION':'str'})


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6097935 entries, 0 to 6097934
Data columns (total 67 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   SEDE                        object 
 1   TIPO_ATENCION               object 
 2   ATENCION                    int64  
 3   DETALLE_ATENCION            object 
 4   CONS_PACIENTE               int64  
 5   INGRESO_PACIENTE            int64  
 6   UBICACION_ATENCION          object 
 7   HABITACION_ATENCION         object 
 8   ESPECIALIDAD_ATENCION       object 
 9   MEDICO_ATENCION             object 
 10  TIPO_RESPON_PAGO            object 
 11  RESPONSABLE_PAGO            object 
 12  TARIFA_ATENCION             object 
 13  TIPO_PACIENTE               object 
 14  SEXO_PACIENTE               object 
 15  FECHA_NACI_PACIENTE         object 
 16  EDAD_INICIO_ATENCION        int64  
 17  EDAD_EGRESO_ATENCION        int64  
 18  FECHA_INGRESO_ATENCION      object 
 19  FECHA_EGRESO_ATENCION

In [10]:
df_origin.head()

,SEDE,TIPO_ATENCION,ATENCION,DETALLE_ATENCION,CONS_PACIENTE,INGRESO_PACIENTE,UBICACION_ATENCION,HABITACION_ATENCION,ESPECIALIDAD_ATENCION,MEDICO_ATENCION,...,TERCERO_CARGO,FUENTE_FACTURA,NUMERO_FACTURA,CANTIDAD_CARGO,VALOR_TOTAL_CARGO,VALOR_FACTURADO_CARGO,CAUSA_RECLAMO,VALOR_RECLAMADO,VALOR_ACEPTADO,VALOR_FINAL_RECLAMADO
0,04101,hosp_urge,771610,HOSP,219474,1,CHUR,NaN,CON,NaN,...,72182094,FH,5452,1,110000,110000,NaN,0.0,0,0
1,04101,hosp_urge,779997,HOSP,221797,1,CHUR,NaN,CON,NaN,...,0,FH,5744,1,76270,76270,NaN,0.0,0,0
2,04101,hosp_urge,781028,HOSP,222039,1,CHHD,J210,387,1085903094,...,0,FH,5489,1,16152,16152,NaN,0.0,0,0
3,04101,hosp_urge,774927,HOSP,220359,1,CHUR,NaN,CON,NaN,...,46669964,FH,5613,1,110000,110000,NaN,0.0,0,0
4,04101,hosp_urge,763117,HOSP,217386,1,CHUR,NaN,CON,NaN,...,52056800,FH,3061,1,58300,58300,NaN,0.0,0,0


In [11]:
df_origin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6097935 entries, 0 to 6097934
Data columns (total 67 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   SEDE                        object 
 1   TIPO_ATENCION               object 
 2   ATENCION                    int64  
 3   DETALLE_ATENCION            object 
 4   CONS_PACIENTE               int64  
 5   INGRESO_PACIENTE            int64  
 6   UBICACION_ATENCION          object 
 7   HABITACION_ATENCION         object 
 8   ESPECIALIDAD_ATENCION       object 
 9   MEDICO_ATENCION             object 
 10  TIPO_RESPON_PAGO            object 
 11  RESPONSABLE_PAGO            object 
 12  TARIFA_ATENCION             object 
 13  TIPO_PACIENTE               object 
 14  SEXO_PACIENTE               object 
 15  FECHA_NACI_PACIENTE         object 
 16  EDAD_INICIO_ATENCION        int64  
 17  EDAD_EGRESO_ATENCION        int64  
 18  FECHA_INGRESO_ATENCION      object 
 19  FECHA_EGRESO_ATENCION

In [12]:
df_origin.duplicated()

0          False
1          False
2          False
3          False
4          False
           ...  
6097930    False
6097931    False
6097932    False
6097933    False
6097934    False
Length: 6097935, dtype: bool

In [13]:
df_origin.isnull().sum()

SEDE                           0
TIPO_ATENCION                  0
ATENCION                       0
DETALLE_ATENCION               0
CONS_PACIENTE                  0
                          ...   
VALOR_FACTURADO_CARGO          0
CAUSA_RECLAMO            6085842
VALOR_RECLAMADO            11973
VALOR_ACEPTADO                 0
VALOR_FINAL_RECLAMADO          0
Length: 67, dtype: int64

In [14]:
df_copy=df_origin.copy()
df_copy.head()

,SEDE,TIPO_ATENCION,ATENCION,DETALLE_ATENCION,CONS_PACIENTE,INGRESO_PACIENTE,UBICACION_ATENCION,HABITACION_ATENCION,ESPECIALIDAD_ATENCION,MEDICO_ATENCION,...,TERCERO_CARGO,FUENTE_FACTURA,NUMERO_FACTURA,CANTIDAD_CARGO,VALOR_TOTAL_CARGO,VALOR_FACTURADO_CARGO,CAUSA_RECLAMO,VALOR_RECLAMADO,VALOR_ACEPTADO,VALOR_FINAL_RECLAMADO
0,04101,hosp_urge,771610,HOSP,219474,1,CHUR,NaN,CON,NaN,...,72182094,FH,5452,1,110000,110000,NaN,0.0,0,0
1,04101,hosp_urge,779997,HOSP,221797,1,CHUR,NaN,CON,NaN,...,0,FH,5744,1,76270,76270,NaN,0.0,0,0
2,04101,hosp_urge,781028,HOSP,222039,1,CHHD,J210,387,1085903094,...,0,FH,5489,1,16152,16152,NaN,0.0,0,0
3,04101,hosp_urge,774927,HOSP,220359,1,CHUR,NaN,CON,NaN,...,46669964,FH,5613,1,110000,110000,NaN,0.0,0,0
4,04101,hosp_urge,763117,HOSP,217386,1,CHUR,NaN,CON,NaN,...,52056800,FH,3061,1,58300,58300,NaN,0.0,0,0


In [15]:
df_copy = df_copy.fillna({'CAUSA_RECLAMO':0})
df_copy.head()

,SEDE,TIPO_ATENCION,ATENCION,DETALLE_ATENCION,CONS_PACIENTE,INGRESO_PACIENTE,UBICACION_ATENCION,HABITACION_ATENCION,ESPECIALIDAD_ATENCION,MEDICO_ATENCION,...,TERCERO_CARGO,FUENTE_FACTURA,NUMERO_FACTURA,CANTIDAD_CARGO,VALOR_TOTAL_CARGO,VALOR_FACTURADO_CARGO,CAUSA_RECLAMO,VALOR_RECLAMADO,VALOR_ACEPTADO,VALOR_FINAL_RECLAMADO
0,04101,hosp_urge,771610,HOSP,219474,1,CHUR,NaN,CON,NaN,...,72182094,FH,5452,1,110000,110000,0.0,0.0,0,0
1,04101,hosp_urge,779997,HOSP,221797,1,CHUR,NaN,CON,NaN,...,0,FH,5744,1,76270,76270,0.0,0.0,0,0
2,04101,hosp_urge,781028,HOSP,222039,1,CHHD,J210,387,1085903094,...,0,FH,5489,1,16152,16152,0.0,0.0,0,0
3,04101,hosp_urge,774927,HOSP,220359,1,CHUR,NaN,CON,NaN,...,46669964,FH,5613,1,110000,110000,0.0,0.0,0,0
4,04101,hosp_urge,763117,HOSP,217386,1,CHUR,NaN,CON,NaN,...,52056800,FH,3061,1,58300,58300,0.0,0.0,0,0
